In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf

import os

#### Thay đổi hiển thị ####
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.set_option('display.max_row', None)
pd.set_option('display.max_columns', None)
#### Change display ####

file_path = ["/mnt/c/Users/hoang/FileCSV_DACN_2025/IoT23.csv", "C:\\Users\\hoang\\FileCSV_DACN_2025\\IoT23.csv"]
out_path =  ["/mnt/c/Users/hoang/FileCSV_DACN_2025/parquet_shuffled_IoT23", "C:\\Users\\hoang\\FileCSV_DACN_2025\\parquet_shuffled_IoT23"]

if os.name == 'nt':
    file_path = file_path[1]
    out_path = out_path[1]
else:
    file_path = file_path[0]
    out_path = out_path[0]

print(file_path)

dictTypes = {}
df = dk.read_csv(file_path)
for col in df.columns:
    if col.startswith('label'):
        dictTypes[col]='str'
    else:
        dictTypes[col]='float32'
del df

df =dk.read_csv(file_path, blocksize= "10MB", dtype=dictTypes)

print(df.npartitions)
print(df.dtypes)

benign_df = df[df['label'] == 'benign']
bash_spr_df= df[ df['label']=='bashlite_spread']
df = df[~df['label'].isin(['benign', 'bashlite_spread'])]

benign_df = benign_df.sample(frac=0.3, random_state=42)
bash_spr_df= bash_spr_df.sample(frac=0.4, random_state=42)

df = dk.concat([benign_df, df])
del benign_df
df = dk.concat([df, bash_spr_df])
del bash_spr_df

print(df.head(20))

C:\Users\hoang\FileCSV_DACN_2025\scaled_medbiot.csv
65
MI_dir_5_weight                     float32
MI_dir_5_mean                       float32
MI_dir_5_std                        float32
MI_dir_3_weight                     float32
MI_dir_3_mean                       float32
MI_dir_3_std                        float32
MI_dir_1_weight                     float32
MI_dir_1_mean                       float32
MI_dir_1_std                        float32
MI_dir_0.1_weight                   float32
MI_dir_0.1_mean                     float32
MI_dir_0.1_std                      float32
MI_dir_0.01_weight                  float32
MI_dir_0.01_mean                    float32
MI_dir_0.01_std                     float32
HH_5_weight_0                       float32
HH_5_mean_0                         float32
HH_5_std_0                          float32
HH_5_radius_0_1                     float32
HH_5_magnitude_0_1                  float32
HH_5_covariance_0_1                 float32
HH_5_pcc_0_1         

In [8]:
from dask_ml.model_selection import train_test_split 
from sklearn.utils import shuffle

# def shuffle_dask_dataframe(df, random_state=None):
#     return (
#         df.assign(__shuffle_key=lambda x: np.random.RandomState(random_state).random(len(x)))
#         .set_index("__shuffle_key", npartitions=df.npartitions)
#         .reset_index(drop=True)
#     )

def shuffle_dask_dataframe(df, random_state=None):
    return (
        df.map_partitions(
            lambda part: part.assign(__shuffle_key=np.random.RandomState(random_state).random(len(part)))
        )
        .shuffle('__shuffle_key')
        .drop(columns='__shuffle_key')
    )

df = shuffle_dask_dataframe(df, 42)

print(df.dtypes)
# print(df.npartitions)

MI_dir_5_weight                     float32
MI_dir_5_mean                       float32
MI_dir_5_std                        float32
MI_dir_3_weight                     float32
MI_dir_3_mean                       float32
MI_dir_3_std                        float32
MI_dir_1_weight                     float32
MI_dir_1_mean                       float32
MI_dir_1_std                        float32
MI_dir_0.1_weight                   float32
MI_dir_0.1_mean                     float32
MI_dir_0.1_std                      float32
MI_dir_0.01_weight                  float32
MI_dir_0.01_mean                    float32
MI_dir_0.01_std                     float32
HH_5_weight_0                       float32
HH_5_mean_0                         float32
HH_5_std_0                          float32
HH_5_radius_0_1                     float32
HH_5_magnitude_0_1                  float32
HH_5_covariance_0_1                 float32
HH_5_pcc_0_1                        float32
HH_3_weight_0                   

In [9]:
print(df.npartitions)

195


In [10]:
# header = True
# for i in range(df.npartitions):
#     df.get_partition(i).compute().to_csv(out_path, mode='a', header = header, index=False)
#     header = False

df.to_parquet(out_path, engine='pyarrow')

# VAEX TIME

In [1]:
import vaex

# Mở file lớn
df = vaex.open('/mnt/c/Users/hoang/FileCSV_DACN_2025/IoT23.csv')

# Shuffle và xuất
df.shuffle().export_hdf5('/mnt/c/Users/hoang/FileCSV_DACN_2025/shuffled_dataset.hdf5')

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.11/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.11/u/import-error

# DASK BUT ...

In [ ]:
import dask.dataframe as dd
from dask.distributed import Client

# Khởi tạo cluster cục bộ
client = Client(n_workers=2, memory_limit='6GB')  # Tùy chỉnh theo máy

# Đọc dữ liệu với kích thước partition phù hợp
ddf = dd.read_csv(
    '/mnt/c/Users/hoang/FileCSV_DACN_2025/IoT23.csv',
    blocksize='256MB',  # Thử nghiệm giá trị này
    )

# Shuffle với repartition trước nếu cần
ddf = ddf.repartition(partition_size='64MB')  # Giảm số partition quá nhỏ

# Thực hiện shuffle
ddf = ddf.sample(frac=1, random_state=42)
# ddf = ddf.repartition(partition_size='2GB')

# Lưu kết quả (nên dùng Parquet thay CSV)
ddf.to_parquet(
    '/mnt/c/Users/hoang/FileCSV_DACN_2025/',
    engine='pyarrow',
    compression='snappy',  # cân bằng giữa tốc độ và tỉ lệ nén
    write_index=False,
    partition_on=['column_name']  # nếu cần partition
)

2025-05-23 16:32:00,701 - distributed.nanny - WARNING - Restarting worker
2025-05-23 16:32:03,471 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.92 GiB -- Worker memory limit: 5.59 GiB
2025-05-23 16:32:11,489 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 4.48 GiB -- Worker memory limit: 5.59 GiB
2025-05-23 16:32:16,433 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.06 GiB -- Worker memory limit: 5.59 GiB
2025-05-23 16:32:17,961 - distributed.nanny - WARN

# CONVERT sang parquet

In [1]:
import dask.dataframe as dd
import os
#Load
file_path= ["/mnt/c/Users/hoang/FileCSV_DACN_2025/IoT23.csv", "C:/Users/hoang/FileCSV_DACN_2025/IoT23.csv"]
out_path = ["/mnt/c/Users/hoang/FileCSV_DACN_2025/parquet_IoT23/","C:/Users/hoang/FileCSV_DACN_2025/parquet_IoT23/" ]
file_path = file_path[1] if os.name=='nt' else file_path[0]
out_path = out_path[1] if os.name =='nt' else out_path[0]

print(file_path)
dictTypes = {}
df = dd.read_csv(file_path)
for col in df.columns:
    if col.startswith('proto') == True:
        dictTypes[col] = 'int32'
    elif col.startswith('service_') == True:
        dictTypes[col] = 'int32'
    # elif col == 'label':
    #     dictTypes[col]= 'int32'
    elif col.startswith('detailed-label'):
        dictTypes[col] = 'str'
    else:
        dictTypes[col]='float32'

df = dd.read_csv(file_path, dtype=dictTypes, blocksize= "256MB")
print(df.npartitions)
print(df.dtypes)
df.to_parquet(out_path, engine='pyarrow')

C:/Users/hoang/FileCSV_DACN_2025/IoT23.csv
168
id.orig_h                 float32
id.orig_p                 float32
id.resp_h                 float32
id.resp_p                 float32
duration                  float32
orig_bytes                float32
resp_bytes                float32
conn_state                float32
missed_bytes              float32
history                   float32
orig_pkts                 float32
orig_ip_bytes             float32
resp_pkts                 float32
resp_ip_bytes             float32
label                     float32
detailed-label    string[pyarrow]
proto_icmp                  int32
proto_tcp                   int32
proto_udp                   int32
service_dhcp                int32
service_dns                 int32
service_http                int32
service_irc                 int32
service_ssh                 int32
service_ssl                 int32
dtype: object


In [8]:
import dask.dataframe as dd
import os
file_path = ["/mnt/c/Users/hoang/FileCSV_DACN_2025/parquet_IoT23/","C:/Users/hoang/FileCSV_DACN_2025/parquet_IoT23/" ]

df = dd.read_parquet(file_path[1]) if os.name =='nt' else dd.read_parquet(file_path[0])
print(df.npartitions)
df.repartition(partition_size="1MB")
print(df.npartitions)
print(df.tail(10))

168
168
         id.orig_h  id.orig_p  id.resp_h  id.resp_p  duration  orig_bytes  \
1815702   3.232261   0.727294   3.565114   0.350872       0.0         0.0   
1815703   3.232261   0.625069   0.892884   0.350872       0.0         0.0   
1815704   3.232261   0.797088   2.446045   0.350872       0.0         0.0   
1815705   3.232261   0.793331   1.135935   0.350872       0.0         0.0   
1815706   3.232261   0.990701   1.122722   0.350872       0.0         0.0   
1815707   3.232261   0.422713   2.908634   0.350872       0.0         0.0   
1815708   3.232261   0.103287   1.170498   0.350872       0.0         0.0   
1815709   3.232261   0.154855   3.229167   0.350872       0.0         0.0   
1815710   2.825261   0.313650   3.232261   0.025953       0.0         0.0   
1815711   3.227416   0.313650   3.232261   0.025953       0.0         0.0   

         resp_bytes  conn_state  missed_bytes   history  ...  \
1815702         0.0    0.722481           0.0  0.223978  ...   
1815703         